# Análise de retornos

## Variação Percentual $\rightarrow$ Retornos Simples

$$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}}
$$

In [ ]:
import pandas as pd
from bcb import currency

In [ ]:
usdbrl = currency.get('USD', start_date='2020-01-01', end_date='2020-01-08')

In [ ]:
usdbrl_ask = usdbrl.loc[:, ('USD', 'ask')]

In [ ]:
P_t_1 = usdbrl_ask.shift(1)
R_t = usdbrl_ask.diff() / P_t_1

In [ ]:
df = pd.concat([R_t, P_t_1, usdbrl_ask], axis=1)\
    .set_axis(['R_t', 'P_t-1', 'P_t'], axis=1)

In [ ]:
df

## Preços em função dos Retornos Simples

$$
P_t = P_{t-1} (1 + R_t)
$$

In [ ]:
df['P_t calc'] = df['P_t-1'] * (1 + df['R_t'])

In [ ]:
df.head()

## Composição dos Retornos Simples

$$
\begin{eqnarray}
P_t & = & P_{t-1} (1 + R_t) \\
P_{t-1} & = & P_{t-2} (1 + R_{t-1}) \\
P_{t-2} & = & P_{t-3} (1 + R_{t-2}) \\
\end{eqnarray}
$$

##### **Escrevendo $P_t$ em termos de $R_{t}$, $R_{t-1}$, $R_{t-2}$**

$$
\begin{eqnarray}
P_{t-2} & = & P_{t-3} (1 + R_{t-2}) \\
P_{t-1} & = & P_{t-3} (1 + R_{t-2}) (1 + R_{t-1}) \\
P_t & = & P_{t-3} (1 + R_{t-2}) (1 + R_{t-1}) (1 + R_t) \\
\end{eqnarray}
$$

##### **Generalizando**

$$
P_t = P_{t-N} \Pi_{i=0}^{N-1} (1 + R_{t-i})
$$

In [ ]:
df.loc['2020-01-02', 'P_t'] * (1 + df['R_t']).prod()

## Reconstruir os Preços a partir dos Retornos

In [ ]:
(1 + df['R_t']).cumprod()

In [ ]:
P_t_recon = df.loc['2020-01-02', 'P_t'] * (1 + df['R_t']).cumprod()

In [ ]:
P_t_recon

In [ ]:
P_t_recon['2020-01-02'] = df.loc['2020-01-02', 'P_t']

In [ ]:
P_t_recon

In [ ]:
pd.concat([df['P_t'], P_t_recon], axis=1)\
    .set_axis(['P_t', 'P_t_recon'], axis=1)

## Curva de Rentabilidade de um Ativo

In [ ]:
rentabilidade = 100 * (1 + df['R_t']).cumprod()

In [ ]:
rentabilidade.iloc[0] = 100

In [ ]:
rentabilidade

## Comparar as Rentabilidades de Diversos Ativos

Para 2020, usar:

- USDBRL
- BOVA11
- SMLL11

In [ ]:
import yfinance as yf

In [ ]:
etfs = yf.download(['BOVA11.SA', 'SMAL11.SA'], start='2020-01-01', end='2020-12-31', progress=False)

In [ ]:
etfs.head()

In [ ]:
etfs = etfs['Adj Close']

In [ ]:
usdbrl = currency.get('USD', start_date='2020-01-01', end_date='2020-12-31')

In [ ]:
usdbrl_ask = usdbrl.loc[:, ('USD', 'ask')]

In [ ]:
series = pd.concat([usdbrl_ask, etfs], axis=1)\
    .set_axis(['USDBRL', 'BOVA11', 'SMAL11'], axis=1)

In [ ]:
series.head()

In [ ]:
R_t = series\
    .apply(lambda x: x.diff() / x.shift(1))

In [ ]:
R_t.head()

In [ ]:
rentabilidade = 100 * R_t.apply(lambda x: (1 + x).cumprod())

In [ ]:
rentabilidade.iloc[0, :] = 100

In [ ]:
rentabilidade.head()

In [ ]:
rentabilidade.plot(figsize=(15,5), grid=True, title='Rentabilidade acumulada');

## Resumo

- Variação Percentual $\longrightarrow$ Retornos Simples
- Reconstruindo Preços a partir dos Retornos Simples: $P_t = P_{t-N} \Pi_{i=0}^{N-1} (1 + R_{t-i})$
- Rentabilidade $\longrightarrow$ Retornos Simples Acumulados: $\Pi_{i=0}^{N-1} (1 + R_{t-i})$